#### Importing Libraries

In [0]:
import pandas as pd
import joblib

#### Loading the trained model

In [0]:
model_path = "linear_regression_churn_model.pkl"

lr_model = joblib.load(model_path)
print("Model loaded successfully.")

encoder = joblib.load('encoder.pkl')

#### Preprocess new data

In [0]:
df_new['ContractType'].unique()

In [0]:
df_new['PaymentMethod'].unique()

In [0]:
import pandas as pd
import joblib

# Load new data
df_new = pd.read_csv('github_churn_10000.csv')

customer_ids = df_new['CustomerID']

# Drop CustomerID from features
X_new_raw = df_new.drop(columns=['CustomerID'])

# Apply the same one-hot encoding to categorical columns
categorical_cols = ['Gender', 'ContractType', 'PaymentMethod']

X = pd.get_dummies(X_new_raw, columns=categorical_cols, drop_first=True)




features_to_scale = ['Age', 'Tenure', 'MonthlyCharges', 'TotalCharges']
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
scaler = StandardScaler()
X[features_to_scale] = scaler.fit_transform(X[features_to_scale])



In [0]:
X

### Predict churn on unseen data 

In [0]:


# Predict
y_pred = lr_model.predict(X)
y_pred_proba = lr_model.predict_proba(X)[:, 1]

#Add predictions to DataFrame
df_new['Churn'] = y_pred
df_new['Churn_Probability'] = y_pred_proba


In [0]:
y_pred

In [0]:
df_new

In [0]:
df_new['Churn_Prediction'].unique()

In [0]:
df_new['Churn_Prediction'].nunique()

#### Store the prediction results

In [0]:
df_new.to_csv('churn_predictions.csv', index=False)
print("Predictions saved to churn_predictions.csv")